In [1]:
# 先import會用到的庫
from pymongo import MongoClient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, KMeans, DBSCAN, estimate_bandwidth
from collections import defaultdict
from datetime import datetime
import time
from tqdm import tqdm

In [2]:
import torch
if torch.cuda.is_available():
    print('CUDA is available')
else:
    print('CUDA is not available')

CUDA is available


In [5]:
### 把business Id重新編碼
client = MongoClient('127.0.0.1', 27017)
db_new = client.Yelp_New
business_new = db_new.business

db_final = client.Yelp_Final
business_final = db_final.business

bar = tqdm(total=business_new.count_documents({}), desc='ReId Business')
tempIds = business_new.find({}, no_cursor_timeout=True, batch_size=10)
tid = 0
for item in tempIds:
    item['newId'] = str(tid)
    business_final.insert_one(item)
    tid += 1
    bar.update(1)
tempIds.close()
bar.close()

ReId Business:   0%|          | 0/12360 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
ReId Business: 100%|██████████| 12360/12360 [00:01<00:00, 6918.04it/s]


In [6]:
### 把user Id重新編碼
client = MongoClient('127.0.0.1', 27017)
db_new = client.Yelp_New
user_new = db_new.user

db_final = client.Yelp_Final
user_final = db_final.user

bar = tqdm(total=user_new.count_documents({}), desc='ReId User')
tempIds = user_new.find({}, no_cursor_timeout=True, batch_size=10)
tid = 0
for item in tempIds:
    item['newId'] = str(tid)
    user_final.insert_one(item)
    tid += 1
    bar.update(1)
tempIds.close()
bar.close()

ReId User:   0%|          | 0/2533 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
ReId User: 100%|██████████| 2533/2533 [00:00<00:00, 6315.65it/s]


In [6]:
### 把user friend重新編碼
client = MongoClient('127.0.0.1', 27017)
db_final = client.Yelp_Final
user_final = db_final.user

def getUserFriends(friends):
    newFriends = []
    sptFriend = friends.replace(' ', '').split(',')
    for friend in sptFriend:
        newFriend = user_final.find_one({'user_id': friend})
        if newFriend is not None:
            newFriends.append(newFriend['newId'])
    return newFriends

bar = tqdm(total=user_final.count_documents({}), desc='ReId User Friend')
tempIds = user_final.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    user_final.update_one({"_id": item['_id']}, {"$set": {'newFriends': getUserFriends(item['friends'])}}, upsert=False)
    bar.update(1)
tempIds.close()
bar.close()

ReId User Friend: 100%|██████████| 1779/1779 [05:20<00:00,  5.55it/s]


In [11]:
### 取得平均情感向量
client = MongoClient('127.0.0.1', 27017)
db_final = client.Yelp_Final
review_final = db_final.review
sentiment_final = db_final.sentiment

sentiment_vectors = {}
bar = tqdm(total=review_final.count_documents({}), desc='Get Average Sentiment')
tempIds = review_final.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    if item['newBusinessId'] not in sentiment_vectors:
        sentiment_vectors[item['newBusinessId']] = np.array(item['sentiment_vector'])
    else:
        sentiment_vectors[item['newBusinessId']] = np.add(sentiment_vectors[item['newBusinessId']], item['sentiment_vector'])
        sentiment_vectors[item['newBusinessId']] = np.divide(sentiment_vectors[item['newBusinessId']], 2)
    bar.update(1)
tempIds.close()
bar.close()

for key, value in sentiment_vectors.items():
    sentiment_final.insert_one({'newBusinessId': key, 'sentiment_vector': value.tolist()})

Get Average Sentiment: 100%|██████████| 61189/61189 [00:09<00:00, 6642.78it/s]


In [7]:
### 刪除newFirends為空的user
client = MongoClient('127.0.0.1', 27017)
db_final = client.Yelp_Final
user_final = db_final.user

bar = tqdm(total=user_final.count_documents({}), desc='Delete User')
tempIds = user_final.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    if len(item['newFriends']) == 0:
        user_final.delete_one({'_id': item['_id']})
    bar.update(1)
tempIds.close()
bar.close()

Delete User: 100%|██████████| 1779/1779 [00:00<00:00, 63693.27it/s]


In [30]:
### 把review的business Id與user Id重新編碼
client = MongoClient('127.0.0.1', 27017)
db_new = client.Yelp_New
review_new = db_new.review_sentiment

db_final = client.Yelp_Final
business_final = db_final.business
user_final = db_final.user
review_final = db_final.review

def getNewId(collection, key, value):
    item = collection.find_one({key: value})
    if item is not None:
        return item['newId']
    return None

bar = tqdm(total=review_new.count_documents({}), desc='ReId Review')
tempIds = review_new.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    businessId = getNewId(business_final, 'business_id', item['business_id'])
    userId = getNewId(user_final, 'user_id', item['user_id'])
    item['newBusinessId'] = businessId
    item['newUserId'] = userId
    if businessId is not None and userId is not None:
        review_final.insert_one(item)
    bar.update(1)
tempIds.close()
bar.close()

ReId Review: 100%|██████████| 77596/77596 [03:27<00:00, 373.98it/s]


In [33]:
### 刪除不在review中的business (因為上面把friends為空的business刪了)
client = MongoClient('localhost', 27017)
db_final = client.Yelp_Final
business_final = db_final.business
review_final = db_final.review

def getIds():
    bar = tqdm(total=business_final.count_documents({}), desc='Delete Business')
    tempIds = business_final.find({}, no_cursor_timeout=True, batch_size=10)
    for item in tempIds:
        checkNeedRemove(item['business_id'])
        bar.update(1)
    tempIds.close()
    bar.close()

def checkNeedRemove(id):
    count = review_final.count_documents({'business_id': id})
    if count == 0:
        business_final.delete_one({'business_id': id})

getIds()

Delete Business:   0%|          | 0/12360 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
Delete Business: 100%|██████████| 12360/12360 [04:00<00:00, 51.45it/s]


In [15]:
### 檢查最後有多少business的review數量不在15~200之間
client = MongoClient('localhost', 27017)
db_final = client.Yelp_Final
business_final = db_final.business
review_final = db_final.review

bar = tqdm(total=business_final.count_documents({}), desc='Delete Business')
tempIds = business_final.find({}, no_cursor_timeout=True, batch_size=10)
needDelete = []
for item in tempIds:
    count = review_final.count_documents({'business_id': item['business_id']})
    if count < 15 or count > 200:
        needDelete.append(item['business_id'])
    bar.update(1)
tempIds.close()
bar.close()

print(len(needDelete))

Delete Business:   0%|          | 0/829 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
Delete Business: 100%|██████████| 829/829 [00:05<00:00, 148.12it/s]

0


In [16]:
### 檢查最後有多少user的review數量不在15~200之間
client = MongoClient('localhost', 27017)
db_final = client.Yelp_Final
user_final = db_final.user
review_final = db_final.review

bar = tqdm(total=user_final.count_documents({}), desc='Delete User')
tempIds = user_final.find({}, no_cursor_timeout=True, batch_size=10)
needDelete = []
for item in tempIds:
    count = review_final.count_documents({'user_id': item['user_id']})
    if count < 15 or count > 200:
        needDelete.append(item['user_id'])
    bar.update(1)
tempIds.close()
bar.close()

print(len(needDelete))

Delete User: 100%|██████████| 1754/1754 [00:14<00:00, 121.04it/s]

1347


In [7]:
### 檢查review表中有幾個unique user_id與business_id
client = MongoClient('localhost', 27017)
db_final = client.Yelp_Final
review_final = db_final.review

userIds = list()
businessIds = list()

def getIds():
    tempIds = review_final.find({}, no_cursor_timeout=True, batch_size=10)
    for item in tempIds:
        userIds.append(item['user_id'])
        businessIds.append(item['business_id'])
    tempIds.close()
    return userIds

userIds = getIds()
print('Business: ' + str(len(np.unique(businessIds))))
print('User: ' + str(len(np.unique(userIds))))

c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


Business: 829
User: 1754


In [3]:
### 複製Final的表到Test
client = MongoClient('localhost', 27017)
db_final = client.Yelp_Final
business_final = db_final.business
review_final = db_final.review
user_final = db_final.user

db_test = client.Yelp_Test1
business_test = db_test.business
review_test = db_test.review
user_test = db_test.user

def copyData(collection, newCollection):
    bar = tqdm(total=collection.count_documents({}), desc='Copy Data')
    tempIds = collection.find({}, no_cursor_timeout=True, batch_size=10)
    for item in tempIds:
        newCollection.insert_one(item)
        bar.update(1)
    tempIds.close()
    bar.close()

copyData(business_final, business_test)
copyData(review_final, review_test)
copyData(user_final, user_test)

Copy Data:   0%|          | 0/11456 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
Copy Data: 100%|██████████| 1779/1779 [00:00<00:00, 5959.01it/s]


In [11]:
### POI分群 by MeanShift
client = MongoClient('localhost', 27017)
db = client.Yelp_Final
business = db.business

# 提取經緯度數據並形成numpy陣列
pois = list(business.find({}, {'newId': 1, 'latitude': 1, 'longitude': 1, '_id': 0}))
coordinates = np.array([[float(poi['latitude']), float(poi['longitude'])] for poi in pois])
bandwidth = estimate_bandwidth(coordinates, quantile=0.001, n_samples=len(coordinates))  # 調整quantile來控制帶寬大小
print('bandwidth: ' + str(bandwidth))

ids = [poi['newId'] for poi in pois]

# 使用MeanShift進行聚類
ms = MeanShift(bandwidth=bandwidth)
ms.fit(coordinates)
labels = ms.labels_

neighbors = defaultdict(list)
for poi_id, label in zip(ids, labels):
    cluster_members = [ids[i] for i in range(len(ids)) if labels[i] == label]
    cluster_members.remove(poi_id)
    neighbors[poi_id] = cluster_members

# 列印結果
# for poi_id, neighbor_ids in neighbors.items():
#     print(poi_id, neighbor_ids)

bar = tqdm(total=business.count_documents({}), desc='Get POI Cluster')
tempIds = business.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    business.update_one({"_id": item['_id']}, {"$set": {'newNeighbors': neighbors[item['newId']]}}, upsert=False)
    bar.update(1)
tempIds.close()
bar.close()

bandwidth: 0.005244939253454207


Get POI Cluster:   0%|          | 0/11456 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
Get POI Cluster: 100%|██████████| 11456/11456 [00:02<00:00, 4356.52it/s]


In [ ]:
### POI分群 by K-means
client = MongoClient('localhost', 27017)
db = client.Yelp_Final
business = db.business

# 提取經緯度數據並形成numpy陣列
pois = list(business.find({}, {'newId': 1, 'latitude': 1, 'longitude': 1, '_id': 0}))
coordinates = np.array([[float(poi['latitude']), float(poi['longitude'])] for poi in pois])
ids = [poi['newId'] for poi in pois]

# 使用KMeans進行聚類，設定K值（聚類數量）
k = 10
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(coordinates)
labels = kmeans.labels_

neighbors = defaultdict(list)
for poi_id, label in zip(ids, labels):
    cluster_members = [ids[i] for i in range(len(ids)) if labels[i] == label]
    cluster_members.remove(poi_id)
    neighbors[poi_id] = cluster_members

# 列印結果
# for poi_id, neighbor_ids in neighbors.items():
#     print(poi_id, neighbor_ids)

bar = tqdm(total=business.count_documents({}), desc='Get POI Cluster')
tempIds = business.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    business.update_one({"_id": item['_id']}, {"$set": {'newNeighbors': neighbors[item['newId']]}}, upsert=False)
    bar.update(1)
tempIds.close()
bar.close()

In [9]:
### POI分群 by DBSCAN
client = MongoClient('localhost', 27017)
db = client.Yelp_Final
business = db.business

# 提取經緯度數據並形成numpy陣列
pois = list(business.find({}, {'newId': 1, 'latitude': 1, 'longitude': 1, '_id': 0}))
coordinates = np.array([[float(poi['latitude']), float(poi['longitude'])] for poi in pois])
ids = [poi['newId'] for poi in pois]

# 使用DBSCAN進行聚類，設定epsilon和最小樣本數
eps = 0.001
min_samples = 3
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(coordinates)
labels = dbscan.labels_

neighbors = defaultdict(list)
for poi_id, label in zip(ids, labels):
    cluster_members = [ids[i] for i in range(len(ids)) if labels[i] == label]
    if poi_id in cluster_members:
        cluster_members.remove(poi_id)
    neighbors[poi_id] = cluster_members

# 列印結果
# for poi_id, neighbor_ids in neighbors.items():
#     print(poi_id, neighbor_ids)

bar = tqdm(total=business.count_documents({}), desc='Get POI Cluster')
tempIds = business.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    business.update_one({"_id": item['_id']}, {"$set": {'newNeighbors': neighbors.get(item['newId'], [])}}, upsert=False)
    bar.update(1)
tempIds.close()
bar.close()

Get POI Cluster:   0%|          | 0/11456 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
Get POI Cluster: 100%|██████████| 11456/11456 [00:21<00:00, 537.84it/s]


In [9]:
### 取得時間特徵
client = MongoClient('localhost', 27017)
db = client.Yelp_Final
review = db.review

def getTimeSlot(dtString, hours_per_slot):
    dtObj = datetime.strptime(dtString, '%d/%m/%Y %H:%M:%S')
    timeSlot = dtObj.hour // hours_per_slot
    return timeSlot

def getTimeProb(newBusinessId, timeSlot):
    total = sum(poiTimeSlot[newBusinessId])
    return poiTimeSlot[newBusinessId][timeSlot] / total

def generate_time_slot_list(hours_per_slot):
    num_slots = 24 // hours_per_slot
    return [0] * num_slots

# 一個區間幾個小時
hours_per_slot = 6
poiTimeSlot = dict()

bar = tqdm(total=review.count_documents({}), desc='Get POI TimeSlot')
tempIds = review.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    if item['newBusinessId'] not in poiTimeSlot:
        poiTimeSlot[item['newBusinessId']] = generate_time_slot_list(hours_per_slot)
    poiTimeSlot[item['newBusinessId']][getTimeSlot(item['date'], hours_per_slot)] += 1
    bar.update(1)
tempIds.close()
bar.close()

bar = tqdm(total=review.count_documents({}), desc='Get POI TimeProb')
tempIds = review.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    timeSlot = getTimeSlot(item['date'], hours_per_slot)
    timeProb = getTimeProb(item['newBusinessId'], timeSlot)
    review.update_one({"_id": item['_id']}, {"$set": {'timeProb': timeProb}}, upsert=False)
    bar.update(1)
tempIds.close()
bar.close()

Get POI TimeSlot:   0%|          | 0/61189 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
Get POI TimeProb: 100%|██████████| 61189/61189 [00:16<00:00, 3637.65it/s]
